In [1]:
# Import modules
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd
import random

In [2]:
# Initiate faker instance
fake = Faker()

In [3]:
# Generate Users
def user_gen(n_user):
    
    domains = ['gmail.com', 'yahoo.com', 'rocketmail.com']
    usernames = [fake.unique.user_name() for _ in range(n_user)]
    emails = [f"{username}@{random.choice(domains)}" for username in usernames]

    # Set date range for the last year
    end_date = datetime(2024, 6, 30)
    start_date = end_date - timedelta(days=365)
    
    users = {
        'user_id': [i + 1 for i in range(n_user)],
        'username': usernames,
        'email': emails
    }

    users_df = pd.DataFrame(users)

    return users_df

In [4]:
# Generate user
users = user_gen(650)

In [5]:
# Generate Libraries
def lib_gen(n_lib):
    
    libraries = {
        'library_id': [f"L-{i:02}" for i in range(1, n_lib + 1)],
        'library_name': [fake.unique.company() for _ in range(n_lib)],
        'location': [fake.country() for _ in range(n_lib)]
    }

    lib_df = pd.DataFrame(libraries)

    return lib_df

In [6]:
# Generate libraries
libraries = lib_gen(8)

In [7]:
# Generate Categories
def category_gen(scraped_data):
    
    read_data = pd.read_csv(scraped_data)
    unique_genres = read_data['genre'].unique()

    categories = {
        'category_id': [f"C-{i:02}" for i in range(1, len(unique_genres) + 1)],
        'category_name': unique_genres
    }
    
    category_df = pd.DataFrame(categories)

    return category_df

In [8]:
# Generate Categories
categories = category_gen('books_data.csv')

In [9]:
# Generate Publishers
def pub_gen(n_pub):
    
    publishers = {
        'publisher_id': [f"P-{i:02}" for i in range(1, n_pub + 1)],
        'publisher_name': [fake.company() for _ in range(n_pub)]
    }

    pub_df = pd.DataFrame(publishers)

    return pub_df

In [10]:
# Generate Publishers
publishers = pub_gen(60)

In [11]:
# Generate Authors
def authors_gen(n_authors):
    
    authors = {
        'author_id': [f"A-{i:02}" for i in range(1, n_authors + 1)],
        'author_name': [fake.name() for _ in range(n_authors)]
    }

    author_df = pd.DataFrame(authors)

    return author_df

In [12]:
# Generate Authors
authors = authors_gen(365)

In [13]:
# Generate Books
def books_gen(scraped_data, lib_df, pub_df, author_df):
    
    # Generate category data and read the scraped book data
    category_df = category_gen(scraped_data)
    read_data = pd.read_csv(scraped_data)

    # Create a mapping of genre to category_id
    genre_to_category_id = dict(zip(category_df['category_name'], category_df['category_id']))

    # Generate book data with correct category_id
    books = {
        'book_id': [f"B-{i:03}" for i in range(1, len(read_data['title']) + 1)],
        'title': read_data['title'].tolist(),  # Use the titles from scraped data
        'category_id': [genre_to_category_id[genre] for genre in read_data['genre']],
        'library_id': [random.choice(lib_df['library_id']) for _ in range(len(read_data['title']))],
        'publisher_id': [random.choice(pub_df['publisher_id']) for _ in range(len(read_data['title']))],
        'author_id': [random.choice(author_df['author_id']) for _ in range(len(read_data['title']))],
        'quantity': [random.randint(1, 10) for _ in range(len(read_data['title']))]
    }

    book_df = pd.DataFrame(books)

    return book_df

In [14]:
# Generate Books
books = books_gen('books_data.csv', libraries, publishers, authors)

In [15]:
# Generate Loans
def loans_gen(n_loan, users_df, book_df, lib_df):
    
    loans = {
        'loan_id': [i + 1 for i in range(n_loan)],
        'user_id': [random.choice(users_df['user_id']) for _ in range(n_loan)],
        'book_id': [random.choice(book_df['book_id']) for _ in range(n_loan)],
        'library_id': [random.choice(lib_df['library_id']) for _ in range(n_loan)],
        'loan_date': [],
        'due_date': [],
        'return_date': []
    }

    # Determine datetime explicitly
    start_date = datetime(2023, 6, 1)
    end_date = datetime(2024, 6, 30)

    # Generate dates
    for _ in range(n_loan):
        loan_date = fake.date_between(start_date=start_date, end_date=end_date)  # Loans from June 2023 to June 2024
        due_date = loan_date + timedelta(days=14)  # 2 weeks loan period
    
        # Randomly decide return timing:
        return_choice = random.choices(
            ["on_time", "late", "not_returned"],
            weights=[0.6, 0.3, 0.1],
            k=1
        )[0]

        if return_choice == "on_time":
            # Returned on time
            return_date = loan_date + timedelta(days=random.randint(1, 14))
        elif return_choice == "late":
            # Returned late within 30 days
            return_date = loan_date + timedelta(days=random.randint(15, 30))
        else:
            # Not returned
            return_date = None

        loans['loan_date'].append(loan_date)
        loans['due_date'].append(due_date)
        loans['return_date'].append(return_date)

    loan_df = pd.DataFrame(loans)

    return loan_df

In [16]:
# Generate Loans
loans = loans_gen(2000, users, books, libraries)

In [17]:
# Generate Holds
def holds_gen(n_hold, users_df, book_df, lib_df):
    
    holds = {
        'hold_id': [],
        'user_id': [],
        'book_id': [],
        'library_id': [],
        'hold_date': [],
        'release_date': [] 
    }

    # Track queue for each books
    book_queue = {book_id: [] for book_id in book_df['book_id']}

    # Define the start and end date for the hold period, ditto, state type explicitly
    start_date = datetime(2023, 6, 1)
    end_date = datetime(2024, 6, 30)

    # Track holds per user
    user_hold_count = {user_id: 0 for user_id in users_df['user_id']}

    for _ in range(n_hold):
        user_id = random.choice(users_df['user_id'])
        
        # Ensure the user has not already held 2 books
        user_holds = [hold for hold in holds['user_id'] if hold == user_id]
        if len(user_holds) >= 2:
            continue  # Skip if the user already holds 2 books

        book_id = random.choice(book_df['book_id'])

        # If the book already has holds, add the user to the queue
        if book_queue[book_id]:
            book_queue[book_id].append(user_id)
            hold_date = fake.date_between_dates(date_start=start_date, date_end=end_date)
            release_date = hold_date + timedelta(weeks=1)  # Book is held for 1 week
        else:
            # If no holds exist, the book can be borrowed immediately
            book_queue[book_id].append(user_id)
            hold_date = fake.date_between_dates(date_start=start_date, date_end=end_date)
            release_date = hold_date + timedelta(weeks=1)  # Book is held for 1 week
        
        # Add the new hold to the holds data
        holds['hold_id'].append(len(holds['hold_id']) + 1)
        holds['user_id'].append(user_id)
        holds['book_id'].append(book_id)
        holds['library_id'].append(random.choice(lib_df['library_id']))
        holds['hold_date'].append(hold_date)
        holds['release_date'].append(release_date)

        # Increment the user's hold count
        user_hold_count[user_id] += 1
    
    hold_df = pd.DataFrame(holds)

    return hold_df

In [18]:
# Generate Holds
holds = holds_gen(500, users, books, libraries)

In [19]:
# CSV converter
def csv_convert(df_tables, filename):

    csv_file = df_tables.to_csv(filename, index=False)
    
    return csv_file

In [23]:
# Generate Outer layer table
# Can be any of the generated dataframes
csv_convert(bk, 'book_authors.csv')